In [108]:
import pymongo
from pymongo import MongoClient
from pandas.io.json import json_normalize
import pandas as pd
import os


In [109]:
client = MongoClient ('localhost', 27017)
db = client['companies']

In [110]:
#Listado de categorias de las compañias
categorias = db.companies.distinct('category_code');
print(list(categorias))

['enterprise', 'web', 'software', 'news', 'network_hosting', 'games_video', 'mobile', 'music', 'social', 'search', 'messaging', 'advertising', 'photo_video', 'security', 'finance', 'ecommerce', 'travel', 'hardware', 'public_relations', 'other', 'real_estate', 'semiconductor', 'analytics', 'health', 'legal', 'sports', 'biotech', 'cleantech', 'education', 'consulting', 'transportation', None, 'hospitality', 'fashion', 'nonprofit', 'nanotech', 'automotive', 'design', 'manufacturing', 'government', 'local', 'medical']


In [111]:
categorias = ['web', 'software', 'games_video', 'social', 'design', 'search']
campos = {'name': 1,
          'number_of_employees': 1,
          'category_code': 1,
          'acquisition': 1,
          'founded_year': 1,
          'funding_rounds': 1,
          'offices.latitude': 1,
          'offices.longitude': 1,
          'offices.city': 1,
          'offices.country_code': 1}

def buscar_startup(categorias, campos):
    datos = db.companies.find({'$and':[{'category_code': {'$in':categorias}},
                                       {'number_of_employees': {'$lt':100}},
                                       {'offices': {'$exists': True, '$not': {'$size': 0}}},
                                       {'deadpooled_year': None},
                                       {'founded_year':{'$gte': 2010}}]},
                              campos)
    return datos

def buscar_big(categorias):
    datos = db.companies.find({'$and':[{'category_code': {'$in':categorias}},
                                       {'number_of_employees': {'$gt': 250}},
                                       {'offices': {'$exists': True, '$not': {'$size': 0}}},
                                       {'deadpooled_year': None},                                       
                                       {'founded_year':{'$gte': 1980}}]},
                              {'name': 1,
                               'number_of_employees': 1,
                               'category_code': 1,
                               'acquisition': 1,
                               'founded_year': 1,
                               'funding_rounds': 1,
                               'offices.latitude': 1,
                               'offices.longitude': 1,
                               'offices.city': 1,
                               'offices.country_code': 1})
    return datos


In [124]:
big_office_data = json_normalize(data = buscar_big(categorias), 
                             record_path ='offices', 
                             meta=['name', 'category_code', 'founded_year', 'number_of_employees']) 
big_office_data.head()

,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees
0,Menlo Park,USA,37.416050,-122.151801,Facebook,social,2004,5299
1,Dublin,IRL,53.344104,-6.267494,Facebook,social,2004,5299
2,New York,USA,40.755716,-73.979247,Facebook,social,2004,5299
3,San Francisco,USA,37.776805,-122.416924,Twitter,social,2006,1300
4,San Jose,USA,37.295005,-121.930035,eBay,web,1995,15000


In [123]:
startup_office_data = json_normalize(data = buscar_startup(categorias, campos), 
                             record_path ='offices', 
                             meta=['name', 'category_code', 'founded_year', 'number_of_employees']) 
startup_office_data.head()

,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees
0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20
1,Berlin,DEU,52.501345,13.410907,headr,web,2012,8
2,Hannover,DEU,NaN,NaN,headr,web,2012,8
3,San Mateo,USA,37.566879,-122.323895,Fixya,web,2013,30
4,Palo Alto,USA,NaN,NaN,Simplicant,software,2012,10


In [134]:
data = pd.concat([startup_office_data, big_office_data])
data = data.reset_index()
data = data.dropna()
data.shape

(195, 9)

In [135]:
#data['coordenadas'] = [{'type': 'Point', 'coordinates': [x, y]} for x, y in zip(data['longitude'], data['latitude'])]

data['coordenadas'] = [[x, y] for x, y in zip(data['longitude'], data['latitude'])]
data.head()

,index,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees,coordenadas
0,0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20,"[-73.985506, 40.757929]"
1,1,Berlin,DEU,52.501345,13.410907,headr,web,2012,8,"[13.4109071, 52.5013449]"
3,3,San Mateo,USA,37.566879,-122.323895,Fixya,web,2013,30,"[-122.323895, 37.566879]"
5,5,Santa Clara,USA,37.760524,-122.387799,Fuzz,games_video,2011,6,"[-122.387799, 37.760524]"
6,6,Van Nuys,USA,40.650291,-74.294395,CollegeConvo,web,2010,2,"[-74.294395, 40.650291]"


In [136]:
data.to_json('oficinas.json', orient="records", lines=True)

In [137]:
os.system('mongoimport --db oficinas --collection oficinas --drop --file oficinas.json')

0

In [138]:
db_ofi = client['oficinas']
db_ofi.oficinas.create_index([('coordenadas', pymongo.GEOSPHERE )])

'coordenadas_2dsphere'

In [280]:
def buscar_cercanas(lng, lat, min_dist_m = 0, max_dist_m = 5000):
    nearLocation = {
        "lng": lng,
        "lat": lat
    }
    busqueda = db_ofi.oficinas.find({
        "coordenadas": {
         "$near": {
           "$geometry": {
              "type": "Point" ,
              "coordinates": [ nearLocation["lng"] , nearLocation["lat"] ]
           },
           '$minDistance': min_dist_m,
           "$maxDistance": max_dist_m, 
         }
       }
        
    },{'latitude': 1,
       'longitude': 1,
       'city': 1,
       'name': 1,
       'number_of_employees': 1,
       '_id': 0})
    
    return busqueda



#query = buscar_cercanas(-73.990869, 40.748368, 0, 10000)


cercanos = [buscar_cercanas(data.iloc[i]['longitude'], data.iloc[i]['latitude']) for i in range(len(data))]    
#resultado = pd.DataFrame(cercanos)
#resultado[2]
temp = []
cantidad = []
for c in cercanos:
    temp.append(list(c))

for t in temp:
    cantidad.append(len(t))

    
maximos = [(v, cantidad.index(v)) for v in cantidad if v > 5]
print(maximos)
df = pd.DataFrame(temp[30])
df


[(7, 0), (15, 3), (7, 0), (7, 0), (15, 3), (7, 0), (16, 27), (16, 27), (6, 30), (8, 31), (6, 30), (7, 0), (16, 27), (7, 0), (16, 27), (16, 27), (16, 27), (6, 30), (16, 27), (6, 30), (7, 0), (16, 27), (6, 30), (16, 27), (7, 0), (6, 30), (16, 27), (7, 0), (16, 27), (16, 27), (16, 27), (16, 27)]


,city,latitude,longitude,name,number_of_employees
0,Sunnyvale,37.418531,-122.025485,Yahoo!,13600
1,Sunnyvale,37.390753,-122.039865,Blue Coat Systems,1400
2,Santa Clara,37.406456,-121.983146,FileMaker,350
3,Mountain View,37.423258,-122.073635,LinkedIn,4800
4,Santa Clara,37.398076,-121.977333,Avaya,18000
5,Redwood City,37.386774,-121.987661,Evernote,280


In [122]:
import folium     
mapa = folium.Map(location=[15, 0], tiles="openstreetmap", zoom_start=3)

for i in range(0,len(data)):
    if  data.iloc[i]['number_of_employees'] >= 250:
        folium.Marker([data.iloc[i]['latitude'], 
                       data.iloc[i]['longitude']], 
                      popup=data.iloc[i]['name'],
                      icon=folium.Icon(color='blue')).add_to(mapa)
    else:
        folium.Marker([data.iloc[i]['latitude'], 
                       data.iloc[i]['longitude']], 
                      popup=data.iloc[i]['name'],
                      icon=folium.Icon(color='green')).add_to(mapa)  
        
        
        
leyenda = '''
    <style type="text/css">
        #leyenda{
            position: fixed;
            z-index: 9999;
            font-size: 1em;
            background-color: #ffffff;
            color: #333333;
            bottom: 10px;
            right: 10px;
            padding: 10px;
            border: 1px solid #333333;
        }
        
        #leyenda .verde{
            color: #71af26;
        }
        
        #leyenda .azul{
            color: #36a5d6;
        }
    </style>
    
     <div id="leyenda">
        <i class="fa fa-map-marker fa-2x verde"></i> Startup <br>
        <i class="fa fa-map-marker fa-2x azul"></i> Big Company
      </div>
     '''
mapa.get_root().html.add_child(folium.Element(leyenda))
mapa.save('map-oficinas.html')


In [56]:
#from IPython.display import HTML
#HTML(filename='map-oficinas.html')